In [1]:
import pandas as pd

In [19]:
"""
Temperature
Emissions, Population
GDP
Green bonds
Environmental taxes
Environmental protection expenditure
Electric vehicles
Greenhouse policies
"""

'\nTemperature\nEmissions\nGDP\nPopulation\nGreen bonds\nEnvironmental taxes\nEnvironmental protection expenditure\nElectric vehicles\nGreenhouse policies\n'

In [13]:
df_temp_change = pd.read_csv("data/Annual_Surface_Temperature_Change.csv")
df_temp_change.head(2)

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,0.234,1.308,0.457,1.101,1.607,1.568,1.580,0.960,0.544,1.421
1,2,Albania,AL,ALB,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,1.568,1.444,1.322,1.665,1.601,1.269,2.146,1.823,1.623,1.682


In [22]:
df_co2 = pd.read_csv("data/owid-co2-data.csv")
df_co2[df_co2['country'] == "Norway"].tail(3)

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
31505,Norway,2019,NOR,5348285.0,NaN,0.722,0.135,42.784,-1.609,-3.624,...,0.234,0.218,0.197,0.129,0.178,0.183,26.98,45.9,6.327,14.787
31506,Norway,2020,NOR,5379836.0,NaN,0.725,0.135,41.196,-1.588,-3.712,...,0.232,0.204,0.189,0.150,0.189,0.181,NaN,NaN,5.911,14.348
31507,Norway,2021,NOR,5403021.0,NaN,0.701,0.130,40.919,-0.277,-0.673,...,0.229,0.365,0.160,0.163,0.181,0.125,NaN,NaN,NaN,NaN


In [37]:
df_gdp = pd.read_csv("data/GDP.csv",delimiter=";")
#Drop last column, as it is no values
df_gdp = df_gdp.iloc[: , :-1]
df_gdp

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.615084e+09,2.727933e+09,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.202235e+09,3.310056e+09,2.496648e+09,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.129059e+10,2.180847e+10,2.370702e+10,2.821004e+10,2.611879e+10,2.968217e+10,...,9.730435e+11,9.839370e+11,1.003679e+12,9.242525e+11,8.823551e+11,1.020647e+12,9.910223e+11,9.975340e+11,9.216459e+11,1.082096e+12
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.990732e+10,2.014640e+10,2.049713e+10,1.913421e+10,1.811656e+10,1.875347e+10,1.805323e+10,1.879945e+10,2.011614e+10,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.040414e+10,1.112789e+10,1.194319e+10,1.267633e+10,1.383837e+10,1.486223e+10,...,7.275704e+11,8.207927e+11,8.649905e+11,7.607345e+11,6.905464e+11,6.837487e+11,7.416899e+11,7.945430e+11,7.844457e+11,8.358084e+11
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.249982e+11,1.334016e+11,1.372444e+11,8.721929e+10,4.984049e+10,6.897276e+10,7.779294e+10,6.930910e+10,5.361907e+10,7.254699e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.163785e+09,6.735731e+09,7.074658e+09,6.295820e+09,6.682833e+09,7.180813e+09,7.878509e+09,7.899879e+09,7.716925e+09,9.007159e+09
262,"Yemen, Rep.",YEM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.540134e+10,4.041524e+10,4.322859e+10,4.244450e+10,3.131783e+10,2.684223e+10,2.160616e+10,2.188761e+10,1.884051e+10,2.106169e+10
263,South Africa,ZAF,GDP (current US$),NY.GDP.MKTP.CD,8.748597e+09,9.225996e+09,9.813996e+09,1.085420e+10,1.195600e+10,1.306899e+10,...,4.344005e+11,4.008860e+11,3.811989e+11,3.467098e+11,3.235855e+11,3.814488e+11,4.048421e+11,3.879346e+11,3.354421e+11,4.199464e+11
264,Zambia,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,1.082857e+09,...,2.550306e+10,2.803724e+10,2.714102e+10,2.125122e+10,2.095841e+10,2.587360e+10,2.631159e+10,2.330867e+10,1.811063e+10,2.120306e+10


In [7]:
# Green Bonds
df_green_bonds = pd.read_csv("data/Green_Bonds.csv")
df_green_bonds.head(2)

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021
0,1,Argentina,AR,ARG,Green Bond Issuances by Country,US Dollars,Refinitiv. Accessed on 2022-03-29; Country aut...,ECFFI,Green Bonds Issuances,"Environment, Climate Change, Financial and Phy...",...,NaN,NaN,NaN,NaN,NaN,0.973917,NaN,NaN,0.179399,0.895660
1,2,Australia,AU,AUS,Green Bond Issuances by Country,US Dollars,Refinitiv. Accessed on 2022-03-29; Country aut...,ECFFI,Green Bonds Issuances,"Environment, Climate Change, Financial and Phy...",...,NaN,NaN,0.548342,0.563401,0.571439,2.665648,2.367815,2.101684,2.438814,5.011424
